In [ ]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Importing data from 1984-2018
historical = pd.read_csv('historical-data.csv')
historical.head()

In [ ]:
historical.describe()


In [ ]:
#importing stats for 2019-20
this_year = pd.read_csv('2020 stats.csv')
this_year.head()

In [ ]:
features= ['team_wins','per', 'fgm', 'fg_perc', '3pm', '3p_perc', 'ft_perc', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'pts', 'ts', 'trb_perc', 'ast_perc', 'usg_perc', 'ws', 'ws_per_48', 'bpm', 'vorp']

In [ ]:
historical.columns

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
x_data = historical[features].to_numpy()
y_data = historical[['vote_share']].to_numpy()
#y_data = y_data.reshape(y_data.shape[0], )

In [ ]:
rf = RandomForestRegressor(n_estimators=150)

In [ ]:
rf.fit(x_data, y_data.ravel())

In [ ]:
sorted_indices = np.argsort(rf.feature_importances_)[::-1]

In [ ]:
for index in sorted_indices:
    print(f"{features[index]}: {rf.feature_importances_[index]}")

In [ ]:
from sklearn.feature_selection import SelectKBest

In [ ]:
kbest = SelectKBest(mutual_info_regression, k=10)
best_features = kbest.fit_transform(x_data,y_data.ravel())



In [ ]:
sorted_indices = np.argsort(kbest.scores_)[::-1]
for index in sorted_indices:
    print(f"{features[index]}: {kbest.scores_[index]}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
historical.columns

In [ ]:
x = historical[features]
y = historical['vote_share']
corrmat = historical.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(historical[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
feat = historical[features]
feat['vote_share'] = historical['vote_share']

In [ ]:
feat.columns

In [ ]:
x = feat[features]
y = feat['vote_share']


In [ ]:
corrmat = feat.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(feat[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, normalize, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

In [ ]:
features

In [ ]:
train_columns = ['team_wins','per', 'trb', 'ast', 'stl', 'blk', 'pts', 'ts', 'usg_perc', 'ws', 'vorp']
label_columns = ['vote_share']

season = 2018

model = RandomForestRegressor(n_estimators = 200, max_depth = 5)
poly_fit = PolynomialFeatures(degree=2, interaction_only = True)


In [ ]:
train_data = historical.loc[historical.season_start!=season]
validation_data = historical.loc[historical.season_start==season]

train_x = train_data[train_columns].to_numpy()
train_y = train_data[label_columns].to_numpy()
train_y = train_y.reshape(train_y.shape[0], )
#train_x = poly_fit.fit_transform(train_x)


validation_x = validation_data[train_columns].to_numpy()
#validation_x = poly_fit.fit_transform(validation_x)
validation_y = validation_data[label_columns].to_numpy()
validation_y = validation_y.reshape(validation_y.shape[0], )

In [ ]:
model.fit(train_x, train_y)
predict_y_RF = model.predict(validation_x)
predict_y_RF

In [ ]:
validation_y

In [ ]:
model1 = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, verbose=1, max_depth = 10)
model1.fit(train_x, train_y)
predict_y = model1.predict(validation_x)
predict_y

In [ ]:
nn = MLPRegressor(max_iter = 350)
nn.fit(train_x, train_y)
preds = nn.predict(validation_x)
preds

In [ ]:
import xgboost as xgb


In [ ]:
xg = xgb.XGBRegressor(objective = 'reg:squarederror', n_estimators = 200, max_depth = 5, learning_rate = 0.001, alpha = 10)

In [ ]:
xg.fit(train_x, train_y)

In [ ]:
preds= xg.predict(validation_x)

In [ ]:
preds

In [ ]:
validation_y

In [ ]:
yearly_predictions = pd.DataFrame(columns = ['Season', 'Random Forest', 'GBR', 'Bagging', 'AdaBoost', 'Real Vote'])

In [ ]:
def predictions (poly_fit = None, scaler = None):
    global yearly_predictions
    #trial_results = pd.DataFrame(columns = ['Season', 'Random Forest', 'GBR', 'Real Vote'])
    unique_seasons = historical.season_start.unique()
    train_columns = ['team_wins','per', 'trb', 'ast', 'stl', 'blk', 'pts', 'ts', 'usg_perc', 'ws', 'vorp']
    label_columns = ['vote_share']

    
    for seasons in unique_seasons:
        #print(seasons)
        RF = RandomForestRegressor(n_estimators=200, n_jobs=-1)
        GBR = GradientBoostingRegressor(learning_rate = 0.01, n_estimators = 200)
        Bagging = BaggingRegressor(n_estimators=200, n_jobs = -1)
        Ada = AdaBoostRegressor(n_estimators = 200, learning_rate = 0.3)
        train_data = historical.loc[historical.season_start!=seasons]
        validation_data = historical.loc[historical.season_start==seasons]
        
        train_x = train_data[train_columns].to_numpy()
        train_y = train_data[label_columns].to_numpy()
        train_y = train_y.reshape(train_y.shape[0], )

        validation_x = validation_data[train_columns].to_numpy()
        validation_y = validation_data[label_columns].to_numpy()
        validation_y = validation_y.reshape(validation_y.shape[0], )
        
        if poly_fit is not None:
            train_x = poly_fit.fit_transform(train_x)
            validation_x = poly_fit.fit_transform(validation_x)
            
        if scaler is not None:
            train_x = scaler.fit_transform(train_x)
            validation_x = scaler.fit_transform(validation_x)
        
        RF.fit(train_x, train_y)
        RF_preds = RF.predict(validation_x)
        
        GBR.fit(train_x, train_y)
        GBR_preds = GBR.predict(validation_x)
        
        Bagging.fit(train_x, train_y)
        Bagging_preds = Bagging.predict(validation_x)
        
        Ada.fit(train_x, train_y)
        Ada_preds = Ada.predict(validation_x)
        
        
        for i in range(len(validation_x)):
            a = np.array([seasons, RF_preds[i], GBR_preds[i], Bagging_preds[i], Ada_preds[i], validation_y[i]])
            #print(validation_y[i])
            a= pd.Series(a, index=['Season', 'Random Forest', 'GBR', 'Bagging', 'AdaBoost', 'Real Vote'])
            yearly_predictions = yearly_predictions.append(a, ignore_index = True )         
    return yearly_predictions
   

In [ ]:
predictions(PolynomialFeatures(3), MinMaxScaler())

In [ ]:
yearly_predictions.head()

In [ ]:
yearly_predictions.tail()

In [ ]:
predictions.to_excel('rf_gbr_bagging_ada.xlsx')

In [ ]:
current = pd.read_csv('2020 stats.csv')
current.head()

In [ ]:
raw_predictions = np.zeros(10,)

In [ ]:
def this_year(model, poly_fit, train, test, scalar=None, should_print =True):
    global raw_predictions
    #raw_predictions = []
    train_columns = ['team_wins','per', 'trb', 'ast', 'stl', 'blk', 'pts', 'ts', 'usg_perc', 'ws', 'vorp']
    label_columns = ['vote_share']
    
    train_x = train[train_columns].to_numpy()
    train_y = train[label_columns].to_numpy()
    
    train_x, train_y = shuffle(train_x, train_y)
    train_y = train_y.reshape(train_y.shape[0], )
    
    test_x = test[train_columns].to_numpy()
    test_x = np.nan_to_num(test_x)
    
    
    if poly_fit is not None:
        train_x = poly_fit.fit_transform(train_x)
        test_x = poly_fit.fit_transform(test_x)
    
    if scaler is not None:
        train_x = scaler.fit_transform(train_x)
        test_x = scaler.fit_transform(test_x)
        
    model.fit(train_x, train_y)
    predict_y = model.predict(test_x)
    print(predict_y.shape)
    raw_predictions = np.add(raw_predictions, predict_y)
    sorted_indices = np.argsort(predict_y)[::-1]
    predictions = predict_y[sorted_indices]

    
    formatted_preds = []
    if should_print:
        print(f"Predictions:")
    for i in range(10):
        if predictions[i] < 0:
            break
        if should_print:
            print(f"{i+1}. {current.iloc[sorted_indices[i]].Player}: {predictions[i]}")
        formatted_preds.append((current.iloc[sorted_indices[i]].Player, predictions[i]))
    return formatted_preds


In [ ]:
this_year(
    model = RandomForestRegressor(n_estimators =100, n_jobs=-1),
    poly_fit = PolynomialFeatures(2, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
    )

In [ ]:
this_year(
    model = RandomForestRegressor(n_estimators =200, n_jobs=-1),
    poly_fit = PolynomialFeatures(2, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
    )

In [ ]:
this_year(
    model = RandomForestRegressor(n_estimators =100, n_jobs=-1),
    poly_fit = PolynomialFeatures(3, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
    )

In [ ]:
this_year(
    model = RandomForestRegressor(n_estimators =200, n_jobs=-1),
    poly_fit = PolynomialFeatures(3, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
    )

In [ ]:
this_year(
    model = GradientBoostingRegressor(n_estimators =100,alpha= 0.9, max_depth = 7, learning_rate=0.1),
    poly_fit = PolynomialFeatures(2, interaction_only = True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
)

In [ ]:
this_year(
    model = GradientBoostingRegressor(n_estimators =300,alpha= 0.9, max_depth = 5),
    poly_fit = PolynomialFeatures(2, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
)

In [ ]:
this_year(
    model = GradientBoostingRegressor(n_estimators =100,alpha= 0.9, max_depth = 5),
    poly_fit = PolynomialFeatures(3, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
)

In [ ]:
this_year(
    model = GradientBoostingRegressor(n_estimators =300,alpha= 0.9, max_depth = 5),
    poly_fit = PolynomialFeatures(3, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
)

In [ ]:
this_year(
    model = AdaBoostRegressor(n_estimators = 50, learning_rate= 1.0),
    poly_fit = PolynomialFeatures(2, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
)

In [ ]:
this_year(
    model = AdaBoostRegressor(n_estimators = 100, learning_rate= 1.0),
    poly_fit = PolynomialFeatures(2, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
)

In [ ]:
this_year(
    model = AdaBoostRegressor(n_estimators = 100, learning_rate= 0.5),
    poly_fit = PolynomialFeatures(2, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
)

In [ ]:
this_year(
    model = AdaBoostRegressor(n_estimators = 100, learning_rate= 1.0),
    poly_fit = PolynomialFeatures(3, interaction_only=True),
    train = historical,
    test = current,
    scalar = MinMaxScaler()
)

In [ ]:
raw_predictions /=12
raw_predictions

In [ ]:
should_print = 1
sorted_indices = np.argsort(raw_predictions)[::-1]
predictions = raw_predictions[sorted_indices]

    
formatted_preds = []
if should_print:
    print(f"Predictions:")
for i in range(10):
    if predictions[i] < 0:
        break
    if should_print:
        print(f"{i+1}. {current.iloc[sorted_indices[i]].Player}: {predictions[i]}")
    formatted_preds.append((current.iloc[sorted_indices[i]].Player, predictions[i]))



In [ ]:
formatted_preds

In [ ]:
names = [item[0].split(" ")[0][0] + ". " + item[0].split(" ")[1] for item in formatted_preds]
names


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

ax.bar([item[0] for item in formatted_preds], [item[1] for item in formatted_preds])
ax.set_title("Average vote share of MVP Candidates based on 12 models", size=18)
ax.set_ylabel("Vote Share", size=14)
_ = ax.set_xticklabels(labels=[item[0] for item in formatted_preds], rotation=45, ha='right')